In [3]:
%pip install scikit-learn matplotlib seaborn pandas tqdm

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("sumithsingh/blood-cell-images-for-cancer-detection")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'blood-cell-images-for-cancer-detection' dataset.
Path to dataset files: /kaggle/input/blood-cell-images-for-cancer-detection


In [10]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import os
import glob
import random
import shutil
from tqdm import tqdm # For a nice progress bar

# Import metrics and plotting libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize

print(f"PyTorch Version: {torch.__version__}")

# ==============================================================================
# 1. CONFIGURATION: UPDATE THESE PATHS
# ==============================================================================

# ---> STEP 1: Update this path to the folder containing your 5 class folders.
DATA_DIRECTORY = path # This is the original dataset path

# ---> STEP 2: Update this path to your trained ResNet-18 .pth or .pt file.
RESNET_WEIGHTS_PATH = '/content/resnet18_blood_cell_best.pth'

# ---> STEP 3: Update this path to your trained MobileNetV2 .pth or .pt file.
MOBILENET_WEIGHTS_PATH = '/content/mobilenet_v2_best.pth'

# --- Dataset specific parameters ---
CLASS_NAMES = ['basophil', 'erythroblast', 'monocyte', 'myeloblast', 'seg_neutrophil']
NUM_CLASSES = len(CLASS_NAMES)
class_map = {i: name for i, name in enumerate(CLASS_NAMES)}

# ==============================================================================
# 2. SETUP (Test Set Creation, Model Loading, Augmentations)
# ==============================================================================

# --- Helper functions from previous steps ---
def create_test_split(base_dir, class_names, test_split_ratio=0.2):
    # Create the test directory in a writable location
    test_dir = '/content/test_dataset'
    if os.path.exists(test_dir):
        print(f"Test directory already exists at {test_dir}.")
        return test_dir

    print(f"Creating a new test directory at {test_dir}...")
    os.makedirs(test_dir, exist_ok=True)

    for class_name in class_names:
        source_class_dir = os.path.join(base_dir, class_name)
        dest_class_dir = os.path.join(test_dir, class_name)
        os.makedirs(dest_class_dir, exist_ok=True)

        images = glob.glob(os.path.join(source_class_dir, '*.jpg'))
        if not images:
            print(f"No images found in {source_class_dir}")
            continue

        random.shuffle(images)
        test_images = images[:int(len(images) * test_split_ratio)]

        print(f"Moving {len(test_images)} images to {dest_class_dir}")
        for img_path in test_images:
            try:
                shutil.copy(img_path, dest_class_dir) # Use copy instead of move to preserve original dataset
            except Exception as e:
                print(f"Error copying {img_path} to {dest_class_dir}: {e}")

    print("Test set created.")
    return test_dir


def get_your_models(num_classes, resnet_path, mobilenet_path):
    print("\nLoading your trained models...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # ResNet-18
    resnet18 = models.resnet18(weights=None)
    resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)
    try:
        resnet18.load_state_dict(torch.load(resnet_path, map_location=device))
        print(f"OK: ResNet-18 weights loaded.")
    except Exception as e: print(f"ERROR loading ResNet-18: {e}")
    # MobileNetV2
    mobilenet_v2 = models.mobilenet_v2(weights=None)
    mobilenet_v2.classifier[1] = nn.Linear(mobilenet_v2.classifier[1].in_features, num_classes)
    try:
        mobilenet_v2.load_state_dict(torch.load(mobilenet_path, map_location=device))
        print(f"OK: MobileNetV2 weights loaded.")
    except Exception as e: print(f"ERROR loading MobileNetV2: {e}")

    resnet18.to(device).eval()
    mobilenet_v2.to(device).eval()
    return resnet18, mobilenet_v2, device

# --- Define Augmentations ---
test_time_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(), transforms.RandomVerticalFlip(),
    transforms.RandomRotation(45), transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.Resize((224, 224)), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])
original_transform = transforms.Compose([
    transforms.Resize((224, 224)), transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

# ==============================================================================
# 3. FULL EVALUATION LOOP
# ==============================================================================

def perform_tta_inference(model, image_path, device, num_augmentations=5):
    """Slightly modified TTA function to return probabilities."""
    image = Image.open(image_path).convert("RGB")
    all_predictions = []
    with torch.no_grad():
        all_predictions.append(torch.nn.functional.softmax(model(original_transform(image).unsqueeze(0).to(device)), dim=1))
        for _ in range(num_augmentations - 1):
            all_predictions.append(torch.nn.functional.softmax(model(test_time_transforms(image).unsqueeze(0).to(device)), dim=1))
    avg_probabilities = torch.mean(torch.cat(all_predictions, dim=0), dim=0)
    return avg_probabilities.cpu().numpy()

def evaluate_models_on_test_set(test_dir, resnet, mobilenet, device, class_names):
    """Iterates through the entire test set to gather predictions."""
    print("\nStarting evaluation on the full test set...")
    all_image_paths = glob.glob(os.path.join(test_dir, '**', '*.jpg'), recursive=True)

    true_labels = []
    resnet_preds, mobilenet_preds = [], []
    resnet_probs, mobilenet_probs = [], []

    # Create a mapping from class name to index
    class_to_idx = {name: i for i, name in enumerate(class_names)}

    for img_path in tqdm(all_image_paths, desc="Evaluating Images"):
        true_label_name = os.path.basename(os.path.dirname(img_path))
        true_labels.append(class_to_idx[true_label_name])

        # Get predictions for both models
        res_prob = perform_tta_inference(resnet, img_path, device)
        mob_prob = perform_tta_inference(mobilenet, img_path, device)

        resnet_probs.append(res_prob)
        mobilenet_probs.append(mob_prob)

        resnet_preds.append(np.argmax(res_prob))
        mobilenet_preds.append(np.argmax(mob_prob))

    return np.array(true_labels), np.array(resnet_preds), np.array(mobilenet_preds), np.array(resnet_probs), np.array(mobilenet_probs)

# ==============================================================================
# 4. METRICS AND VISUALIZATION FUNCTIONS
# ==============================================================================

def display_metrics(true_labels, pred_labels, pred_probs, class_names, model_name="Model"):
    """Calculates and prints all key metrics."""
    print(f"\n{'='*30}\n📊 Metrics for {model_name}\n{'='*30}")

    # Accuracy
    accuracy = accuracy_score(true_labels, pred_labels)
    print(f"Accuracy: {accuracy:.4f}\n")

    # Classification Report (Precision, Recall, F1-Score)
    print("Classification Report:")
    report = classification_report(true_labels, pred_labels, target_names=class_names)
    print(report)

    # ROC AUC Score
    roc_auc = roc_auc_score(true_labels, pred_probs, multi_class='ovr')
    print(f"ROC AUC Score (One-vs-Rest): {roc_auc:.4f}\n")

    # Confusion Matrix
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix - {model_name}', fontsize=16)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

def plot_metrics_comparison(true_labels, resnet_preds, mobilenet_preds, class_names):
    """Creates a bar chart comparing the two models."""
    metrics = {'Model': [], 'Accuracy': [], 'F1-Score (Macro)': [], 'Recall (Macro)': []}

    for model_name, preds in [('ResNet-18', resnet_preds), ('MobileNetV2', mobilenet_preds)]:
        report = classification_report(true_labels, preds, target_names=class_names, output_dict=True)
        metrics['Model'].append(model_name)
        metrics['Accuracy'].append(report['accuracy'])
        metrics['F1-Score (Macro)'].append(report['macro avg']['f1-score'])
        metrics['Recall (Macro)'].append(report['macro avg']['recall'])

    df = pd.DataFrame(metrics)
    df.plot(x='Model', y=['Accuracy', 'F1-Score (Macro)', 'Recall (Macro)'], kind='bar', figsize=(12, 7), rot=0)
    plt.title('Model Performance Comparison', fontsize=16)
    plt.ylabel('Score')
    plt.ylim(0, 1)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

def plot_roc_curves(true_labels, resnet_probs, mobilenet_probs, class_names):
    """Plots ROC curves for each class for both models."""
    true_labels_bin = label_binarize(true_labels, classes=range(len(class_names)))

    plt.figure(figsize=(12, 9))

    # Plot for ResNet
    for i, class_name in enumerate(class_names):
        fpr, tpr, _ = roc_curve(true_labels_bin[:, i], resnet_probs[:, i])
        plt.plot(fpr, tpr, linestyle='-', label=f'ResNet-18 - {class_name}')

    # Plot for MobileNet
    for i, class_name in enumerate(class_names):
        fpr, tpr, _ = roc_curve(true_labels_bin[:, i], mobilenet_probs[:, i])
        plt.plot(fpr, tpr, linestyle='--', label=f'MobileNetV2 - {class_name}')

    plt.plot([0, 1], [0, 1], 'k:', label='Random Chance')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate (Recall)')
    plt.title('Multi-Class ROC Curves', fontsize=16)
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

# ==============================================================================
# 5. MAIN EXECUTION BLOCK
# ==============================================================================

if __name__ == "__main__":
    if any(p.startswith('path/to/') for p in [DATA_DIRECTORY, RESNET_WEIGHTS_PATH, MOBILENET_WEIGHTS_PATH]):
        print("ERROR: Please update the placeholder paths in the 'CONFIGURATION' section before running.")
    else:
        # Step 1: Setup environment
        test_dir = create_test_split(DATA_DIRECTORY, CLASS_NAMES)
        resnet_model, mobilenet_model, device = get_your_models(NUM_CLASSES, RESNET_WEIGHTS_PATH, MOBILENET_WEIGHTS_PATH)

        # Step 2: Run evaluation on the entire test set
        true_labels, resnet_preds, mobilenet_preds, resnet_probs, mobilenet_probs = evaluate_models_on_test_set(
            test_dir, resnet_model, mobilenet_model, device, CLASS_NAMES
        )

        # Step 3: Display detailed metrics and confusion matrix for each model
        display_metrics(true_labels, resnet_preds, resnet_probs, CLASS_NAMES, model_name="ResNet-18")
        display_metrics(true_labels, mobilenet_preds, mobilenet_probs, CLASS_NAMES, model_name="MobileNetV2")

        # Step 4: Display comparison visualizations
        plot_metrics_comparison(true_labels, resnet_preds, mobilenet_preds, CLASS_NAMES)
        plot_roc_curves(true_labels, resnet_probs, mobilenet_probs, CLASS_NAMES)

        print("\nEvaluation complete.")

PyTorch Version: 2.8.0+cu126
Test directory already exists at /content/test_dataset.

Loading your trained models...
OK: ResNet-18 weights loaded.
OK: MobileNetV2 weights loaded.

Starting evaluation on the full test set...


Evaluating Images:   5%|▍         | 47/1000 [00:36<12:19,  1.29it/s]


KeyboardInterrupt: 

In [11]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
import numpy as np
import os
import glob
import random
import shutil
from tqdm import tqdm # For a nice progress bar

# Import metrics and plotting libraries
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score, roc_curve
from sklearn.preprocessing import label_binarize
from torch.utils.data import Dataset, DataLoader # Import Dataset and DataLoader

print(f"PyTorch Version: {torch.__version__}")

# ==============================================================================
# 1. CONFIGURATION: UPDATE THESE PATHS
# ==============================================================================

# ---> STEP 1: Update this path to the folder containing your 5 class folders.
DATA_DIRECTORY = path # This is the original dataset path

# ---> STEP 2: Update this path to your trained ResNet-18 .pth or .pt file.
RESNET_WEIGHTS_PATH = '/content/resnet18_blood_cell_best.pth'

# ---> STEP 3: Update this path to your trained MobileNetV2 .pth or .pt file.
MOBILENET_WEIGHTS_PATH = '/content/mobilenet_v2_best.pth'

# --- Dataset specific parameters ---
CLASS_NAMES = ['basophil', 'erythroblast', 'monocyte', 'myeloblast', 'seg_neutrophil']
NUM_CLASSES = len(CLASS_NAMES)
class_map = {i: name for i, name in enumerate(CLASS_NAMES)}

# --- Optimization Parameters ---
BATCH_SIZE = 32 # Increase batch size for faster processing
NUM_TTA_AUGMENTATIONS = 5 # Reduced number of TTA augmentations
MAX_TEST_IMAGES_PER_CLASS = 100 # Maximum number of images to test per class

# ==============================================================================
# 2. SETUP (Test Set Creation, Model Loading, Augmentations, Dataset, DataLoader)
# ==============================================================================

# --- Helper functions from previous steps ---
def create_test_split(base_dir, class_names, test_split_ratio=0.2, max_images_per_class=None):
    # Create the test directory in a writable location
    test_dir = '/content/test_dataset'
    if os.path.exists(test_dir):
        print(f"Test directory already exists at {test_dir}.")
        # If the test directory already exists, check if it has the desired number of images
        # If not, remove and recreate it.
        recreate_test_dir = False
        if max_images_per_class:
            for class_name in class_names:
                dest_class_dir = os.path.join(test_dir, class_name)
                if not os.path.exists(dest_class_dir) or len(glob.glob(os.path.join(dest_class_dir, '*.jpg'))) < max_images_per_class:
                    recreate_test_dir = True
                    break
        else: # If max_images_per_class is None, check against test_split_ratio
             # This part would require more complex logic to check the exact split ratio,
             # so for simplicity, we'll just assume if max_images_per_class is None,
             # and the directory exists, it's from a previous run with the split ratio.
             # A more robust solution might involve storing metadata about the split.
             pass # Keep existing test directory if max_images_per_class is not specified and directory exists


        if recreate_test_dir:
            print(f"Existing test directory at {test_dir} does not match the required size. Removing and recreating...")
            shutil.rmtree(test_dir)
        else:
            print(f"Using existing test directory at {test_dir}.")
            return test_dir


    print(f"Creating a new test directory at {test_dir}...")
    os.makedirs(test_dir, exist_ok=True)

    for class_name in class_names:
        source_class_dir = os.path.join(base_dir, class_name)
        dest_class_dir = os.path.join(test_dir, class_name)
        os.makedirs(dest_class_dir, exist_ok=True)

        images = glob.glob(os.path.join(source_class_dir, '*.jpg'))
        if not images:
            print(f"No images found in {source_class_dir}")
            continue

        random.shuffle(images)
        if max_images_per_class:
            test_images = images[:min(len(images), max_images_per_class)]
        else:
            test_images = images[:int(len(images) * test_split_ratio)]


        print(f"Moving {len(test_images)} images to {dest_class_dir}")
        for img_path in test_images:
            try:
                shutil.copy(img_path, dest_class_dir) # Use copy instead of move to preserve original dataset
            except Exception as e:
                print(f"Error copying {img_path} to {dest_class_dir}: {e}")

    print("Test set created.")
    return test_dir


def get_your_models(num_classes, resnet_path, mobilenet_path):
    print("\nLoading your trained models...")
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    # ResNet-18
    resnet18 = models.resnet18(weights=None)
    resnet18.fc = nn.Linear(resnet18.fc.in_features, num_classes)
    try:
        resnet18.load_state_dict(torch.load(resnet_path, map_location=device))
        print(f"OK: ResNet-18 weights loaded.")
    except Exception as e: print(f"ERROR loading ResNet-18: {e}")
    # MobileNetV2
    mobilenet_v2 = models.mobilenet_v2(weights=None)
    mobilenet_v2.classifier[1] = nn.Linear(mobilenet_v2.classifier[1].in_features, num_classes)
    try:
        mobilenet_v2.load_state_dict(torch.load(mobilenet_path, map_location=device))
        print(f"OK: MobileNetV2 weights loaded.")
    except Exception as e: print(f"ERROR loading MobileNetV2: {e}")

    resnet18.to(device).eval()
    mobilenet_v2.to(device).eval()
    return resnet18, mobilenet_v2, device

# --- Define Augmentations ---
# Apply original transform to the PIL Image and return a tensor
original_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# Apply TTA transforms to the PIL Image and return a tensor
test_time_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomRotation(45),
    transforms.ColorJitter(brightness=0.3, contrast=0.3),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


# --- Custom Dataset for Test Set ---
class BloodCellDataset(Dataset):
    def __init__(self, data_dir, class_names, transform=None):
        self.data_dir = data_dir
        self.class_names = class_names
        self.transform = transform
        self.image_paths = []
        self.labels = []
        self.class_to_idx = {name: i for i, name in enumerate(self.class_names)}

        for class_name in class_names:
            class_dir = os.path.join(self.data_dir, class_name)
            for img_path in glob.glob(os.path.join(class_dir, '*.jpg')):
                self.image_paths.append(img_path)
                self.labels.append(self.class_to_idx[class_name])

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        img_path = self.image_paths[idx]
        image = Image.open(img_path).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label # Return image as tensor, and label


# ==============================================================================
# 3. FULL EVALUATION LOOP
# ==============================================================================


def perform_tta_inference_batch(model, images, device):
    """Performs TTA inference on a batch of images (tensors)."""
    model.eval()
    with torch.no_grad():
        all_predictions = []

        # The input 'images' are already tensors with original_transform applied by the DataLoader

        # Apply TTA transforms and get predictions
        # NOTE: Applying random transforms to a batch of tensors requires iterating through the batch.
        # For true batch TTA, you would need transforms that operate on tensors.
        # As a simplification, we'll apply TTA transforms image by image here.
        # A more optimized approach might use a custom collate_fn that applies TTA.
        batch_predictions = []
        for img_tensor in images:
            single_image_predictions = []
            single_image_predictions.append(torch.nn.functional.softmax(model(img_tensor.unsqueeze(0).to(device)), dim=1))
            for _ in range(NUM_TTA_AUGMENTATIONS - 1):
                # Apply TTA transforms to the single image tensor
                augmented_img_tensor = test_time_transforms(transforms.ToPILImage()(img_tensor.cpu())).to(device) # Convert back to PIL for TTA transforms
                single_image_predictions.append(torch.nn.functional.softmax(model(augmented_img_tensor.unsqueeze(0)), dim=1))
            batch_predictions.append(torch.mean(torch.cat(single_image_predictions, dim=0), dim=0))

        avg_probabilities = torch.stack(batch_predictions, dim=0)

        return avg_probabilities.cpu().numpy()


def evaluate_models_on_test_set(test_dir, resnet, mobilenet, device, class_names, batch_size=BATCH_SIZE):
    """Iterates through the entire test set to gather predictions using DataLoader."""
    print("\nStarting evaluation on the full test set...")

    # Use the custom dataset with the transform in __getitem__
    test_dataset = BloodCellDataset(test_dir, class_names, transform=original_transform)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2) # Use multiple workers

    true_labels = []
    resnet_preds, mobilenet_preds = [], []
    resnet_probs, mobilenet_probs = [], []

    for images, labels in tqdm(test_loader, desc="Evaluating Batches"):
        true_labels.extend(labels.numpy())

        # Get predictions for both models using batch TTA
        res_probs_batch = perform_tta_inference_batch(resnet, images, device)
        mob_probs_batch = perform_tta_inference_batch(mobilenet, images, device)

        resnet_probs.extend(res_probs_batch)
        mobilenet_probs.extend(mob_probs_batch)

        resnet_preds.extend(np.argmax(res_probs_batch, axis=1))
        mobilenet_preds.extend(np.argmax(mob_probs_batch, axis=1))

    return np.array(true_labels), np.array(resnet_preds), np.array(mobilenet_preds), np.array(resnet_probs), np.array(mobilenet_probs)

# ==============================================================================
# 4. METRICS AND VISUALIZATION FUNCTIONS
# ==============================================================================

def display_metrics(true_labels, pred_labels, pred_probs, class_names, model_name="Model"):
    """Calculates and prints all key metrics."""
    print(f"\n{'='*30}\n📊 Metrics for {model_name}\n{'='*30}")

    # Accuracy
    accuracy = accuracy_score(true_labels, pred_labels)
    print(f"Accuracy: {accuracy:.4f}\n")

    # Classification Report (Precision, Recall, F1-Score)
    print("Classification Report:")
    report = classification_report(true_labels, pred_labels, target_names=class_names)
    print(report)

    # ROC AUC Score
    roc_auc = roc_auc_score(true_labels, pred_probs, multi_class='ovr')
    print(f"ROC AUC Score (One-vs-Rest): {roc_auc:.4f}\n")

    # Confusion Matrix
    cm = confusion_matrix(true_labels, pred_labels)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title(f'Confusion Matrix - {model_name}', fontsize=16)
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

def plot_metrics_comparison(true_labels, resnet_preds, mobilenet_preds, class_names):
    """Creates a bar chart comparing the two models."""
    metrics = {'Model': [], 'Accuracy': [], 'F1-Score (Macro)': [], 'Recall (Macro)': []}

    for model_name, preds in [('ResNet-18', resnet_preds), ('MobileNetV2', mobilenet_preds)]:
        report = classification_report(true_labels, preds, target_names=class_names, output_dict=True)
        metrics['Model'].append(model_name)
        metrics['Accuracy'].append(report['accuracy'])
        metrics['F1-Score (Macro)'].append(report['macro avg']['f1-score'])
        metrics['Recall (Macro)'].append(report['macro avg']['recall'])

    df = pd.DataFrame(metrics)
    df.plot(x='Model', y=['Accuracy', 'F1-Score (Macro)', 'Recall (Macro)'], kind='bar', figsize=(12, 7), rot=0)
    plt.title('Model Performance Comparison', fontsize=16)
    plt.ylabel('Score')
    plt.ylim(0, 1)
    plt.grid(axis='y', linestyle='--', alpha=0.7)
    plt.show()

def plot_roc_curves(true_labels, resnet_probs, mobilenet_probs, class_names):
    """Plots ROC curves for each class for both models."""
    true_labels_bin = label_binarize(true_labels, classes=range(len(class_names)))

    plt.figure(figsize=(12, 9))

    # Plot for ResNet
    for i, class_name in enumerate(class_names):
        fpr, tpr, _ = roc_curve(true_labels_bin[:, i], resnet_probs[:, i])
        plt.plot(fpr, tpr, linestyle='-', label=f'ResNet-18 - {class_name}')

    # Plot for MobileNet
    for i, class_name in enumerate(class_names):
        fpr, tpr, _ = roc_curve(true_labels_bin[:, i], mobilenet_probs[:, i])
        plt.plot(fpr, tpr, linestyle='--', label=f'MobileNetV2 - {class_name}')

    plt.plot([0, 1], [0, 1], 'k:', label='Random Chance')
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate (Recall)')
    plt.title('Multi-Class ROC Curves', fontsize=16)
    plt.legend(loc='best')
    plt.grid(True)
    plt.show()

# ==============================================================================
# 5. MAIN EXECUTION BLOCK
# ==============================================================================

if __name__ == "__main__":
    if any(p.startswith('path/to/') for p in [DATA_DIRECTORY, RESNET_WEIGHTS_PATH, MOBILENET_WEIGHTS_PATH]):
        print("ERROR: Please update the placeholder paths in the 'CONFIGURATION' section before running.")
    else:
        # Step 1: Setup environment
        test_dir = create_test_split(DATA_DIRECTORY, CLASS_NAMES, max_images_per_class=MAX_TEST_IMAGES_PER_CLASS)
        resnet_model, mobilenet_model, device = get_your_models(NUM_CLASSES, RESNET_WEIGHTS_PATH, MOBILENET_WEIGHTS_PATH)

        # Step 2: Run evaluation on the entire test set
        true_labels, resnet_preds, mobilenet_preds, resnet_probs, mobilenet_probs = evaluate_models_on_test_set(
            test_dir, resnet_model, mobilenet_model, device, CLASS_NAMES
        )

        # Step 3: Display detailed metrics and confusion matrix for each model
        display_metrics(true_labels, resnet_preds, resnet_probs, CLASS_NAMES, model_name="ResNet-18")
        display_metrics(true_labels, mobilenet_preds, mobilenet_probs, CLASS_NAMES, model_name="MobileNetV2")

        # Step 4: Display comparison visualizations
        plot_metrics_comparison(true_labels, resnet_preds, mobilenet_preds, CLASS_NAMES)
        plot_roc_curves(true_labels, resnet_probs, mobilenet_probs, CLASS_NAMES)

        print("\nEvaluation complete.")

PyTorch Version: 2.8.0+cu126
Test directory already exists at /content/test_dataset.
Using existing test directory at /content/test_dataset.

Loading your trained models...
OK: ResNet-18 weights loaded.
OK: MobileNetV2 weights loaded.

Starting evaluation on the full test set...


Evaluating Batches:  44%|████▍     | 14/32 [05:59<07:42, 25.71s/it]

KeyboardInterrupt

